# 1. imports

In [59]:
%%capture
from __future__ import division
import numpy as np
import pandas as pd
import scipy.stats as st
import itertools
from tqdm import tqdm as tqdm
from collections import defaultdict
from collections import Counter

import xmltodict
import cv2
import PIL.Image as pil

import pickle
import boto3
import json
import os
from copy import deepcopy

from IPython.core.display import HTML

from vizier.vizier import Vizier

import os
import random
import datetime

pd.options.mode.chained_assignment = None

%load_ext autoreload
%autoreload 2

# 2. Submitting HITs

In [118]:
task_params = {
    'Title': "Viconary: summarize a video with drawings",
    'Description': "Viconary is a challenge where the player uses the provided icons to draw the sequence of the main activities shown in the video. (1-4 activities)",
    'Keywords': "video, icons, drawing",
    'frame_height': 1170,
    'Reward': f'{0.60}',
    'AssignmentDurationInSeconds': 3600 * 1,
    'AutoApprovalDelayInSeconds': 3600 * 24 * 2,
    'LifetimeInSeconds': 3600 * 24 * 3,
    'MaxAssignments': 1,
}

template_params = {
    'template_file': 'vid2pic_template.html', 
    'template_dir': 'hit_templates' 
}

client_params = {
    'profile_name': 'mturk_vision',
    'in_production': True,
    'n_threads': 4,
}

vizier = Vizier(**client_params)

Account balance is: $1315.22


In [89]:
qual_params = {
    'high_accept_rate': 95,
    'min_total_hits_approved': 1000,
    'english_speaking': ['US', 'CA', 'AU', 'NZ', 'GB'],
    'us_only': ['US'],
    'master': 'False'
}

In [90]:
def _build_qualifications(self, locales=None):
    """
    builds qualifications for task
    :param locales: AMT country codes allowed to perform task
    :return: list of qualification dicts
    """
    if locales:
        locales = [{'Country': loc} for loc in locales]
    masters_id = '2F1QJWKUDD8XADTFD2Q0G6UTO95ALH' if self.in_production else '2ARFPLSP75KLA8M8DH1HTEQVJT3SY6'
    master = {
        'QualificationTypeId': masters_id,
        'Comparator': 'EqualTo',
        'RequiredToPreview': True,
    }
    high_accept_rate = {
        'QualificationTypeId': '000000000000000000L0',
        'Comparator': 'GreaterThanOrEqualTo',
        'IntegerValues': [self.qualifications['high_accept_rate']],
        'RequiredToPreview': True,
    }
    min_total_hits_approved = {
        'QualificationTypeId': '00000000000000000040',
        'Comparator': 'GreaterThanOrEqualTo',
        'IntegerValues': [self.qualifications['min_total_hits_approved']],
        'RequiredToPreview': True,
    }
    location_based = {
        'QualificationTypeId': '00000000000000000071',
        'Comparator': 'In',
        'LocaleValues': locales,
        'RequiredToPreview': True,
    }
    iconary = {
        'QualificationTypeId': '3Z1HL5WC7LSXUFW49BUXR7ZP1IDH6M',
        'Comparator': 'EqualTo',
        'ActionsGuarded': 'DiscoverPreviewAndAccept',
        'IntegerValues': [1]
    }
    iconary_2 = {
        'QualificationTypeId': '3L4WKCMPYDW5XOAL3OFNEQCBWU222H',
        'Comparator': 'EqualTo',
        # 'Comparator': 'DoesNotExist',
        'ActionsGuarded': 'DiscoverPreviewAndAccept',
        'IntegerValues': [1]
    }
    return [high_accept_rate, location_based, iconary_2]

In [91]:
hit_batch_id = 'vic_prod_3_1000'
hit_batch_id = 'vic_prod_4_1000'
hit_batch_id = 'vic_prod_5_2000'

In [119]:
def gen_task_template_args(vid):
    return {'video_id': vid, 'hit_batch_id': hit_batch_id}

In [121]:
train_vids_filename = 'Charades_v1_train_crops_urls.pickle'
test_vids_filename = 'Charades_v1_test_crops_urls.pickle'

with open(train_vids_filename, 'rb') as train_urls:
    train_vids = pickle.load(train_urls)
    train_vids = list(train_vids.values())
with open(test_vids_filename, 'rb') as test_urls:
    test_vids = pickle.load(test_urls)
    test_vids = list(test_vids.values())

vids_to_submit = train_vids[5300:7300]
len(vids_to_submit)

2000

In [122]:
vizier.preview_hit_interface(template_params, **gen_task_template_args(vids_to_submit[0]))

In [123]:
_ = vizier.expected_cost(vids_to_submit, basic_hit_params=task_params)

Insufficient funds: will cost $1440.00 but only $1315.22 available.


just did 3300-4300

just did 4300-5300

make sure to do ~1k test at some point

In [97]:
# %%time 
# created_hits = vizier.create_hit_group(vids_to_submit, gen_task_template_args, basic_hit_params=task_params, template_params=template_params)

# 3. Retreive results

In [124]:
client_params = {
    'profile_name': 'mturk_vision',
    'in_production': True,
    'n_threads': 10,
    's3_base_path': '',
}
vizier_retriever = Vizier(**client_params)

Account balance is: $3315.22


In [125]:
submitted_hits = deepcopy(created_hits)
# submitted_hits = vizier_retriever.unpickle_this('./submitted_batch_production_1000_wed_aug_29_14_34_36_2018.pkl')
hit_batch = [h['HIT'] for h in submitted_hits if h]
# hit_batch = all_sbx_hits

In [126]:
len(hit_batch)

1000

In [127]:
%%time
latest_res = vizier_retriever.get_assignments(hit_batch)

CPU times: user 2.56 s, sys: 205 ms, total: 2.77 s
Wall time: 12.4 s


In [128]:
assignement_stats = pd.Series([h['Assignments'][0]['AssignmentStatus'] for h in latest_res if h['Assignments']])

In [129]:
assignement_stats.value_counts()

Submitted    634
dtype: int64

In [116]:
len(latest_res)

1000

In [117]:
assignment_batch = [a for r in latest_res for a in r['Assignments']]
len(assignment_batch)

582

In [138]:
# _resp = vizier_retriever.approve_assignments(latest_res)

In [100]:
hss = {}
for h in tqdm(hit_batch):
    hid = h['HITId']
    hr = vizier.amt.client.get_hit(HITId=hid)
    hs = hr['HIT']['HITStatus']
    hss[hid] = hs

pd.Series(list(hss.values())).value_counts()

assignable_hits = [{'HITId': k} for k, stat in hss.items() if stat=='Assignable']

100%|██████████| 200/200 [00:27<00:00,  7.39it/s]


In [159]:
assignement_returned_df = pd.DataFrame(assignment_batch)

In [160]:
assignement_returned_df.to_pickle('hitdata.pkl')

In [102]:
pd.Series(list(hss.values())).value_counts()

Reviewable    200
dtype: int64

In [104]:
# k, stat in hss.items():
#     if stat='Assignable':
#         print('expiring')
#         vizier.amt.client.

In [17]:
hr = vizier.amt.client.get_hit(HITId=h1)

In [69]:
dupe_aids = ['38JBBYETQPXHUECAYCGWMFTJFXD4EM', '3P1L2B7AD2CZ4DWKYII9D8P85M1LOP']

In [81]:
all_submitted_aids = [r['Assignments'][0]['AssignmentId'] for r in latest_res]

In [83]:
pd.Series(all_submitted_aids).to_pickle('submitted_aids.pkl')

In [79]:
for r in latest_res:
    if r['Assignments'][0]['AssignmentId'] in dupe_aids:
        print(r)

{'NextToken': 'p1:Fw3fF6gYtYJjQTgCKn58p4G6XE8PaNNpQnwCB6E71TlVeSzHm7GUoP6qyq86Bw==', 'NumResults': 1, 'Assignments': [{'AssignmentId': '3P1L2B7AD2CZ4DWKYII9D8P85M1LOP', 'WorkerId': 'A8TLS2F8LFQ68', 'HITId': '34KYK9TV2RVZ3D47OK5MUQ7J97LSBO', 'AssignmentStatus': 'Submitted', 'AutoApprovalTime': datetime.datetime(2018, 7, 20, 5, 23, 3, tzinfo=tzlocal()), 'AcceptTime': datetime.datetime(2018, 7, 18, 5, 11, 35, tzinfo=tzlocal()), 'SubmitTime': datetime.datetime(2018, 7, 18, 5, 23, 3, tzinfo=tzlocal()), 'Answer': '<?xml version="1.0" encoding="ASCII"?><QuestionFormAnswers xmlns="http://mechanicalturk.amazonaws.com/AWSMechanicalTurkDataSchemas/2005-10-01/QuestionFormAnswers.xsd"><Answer><QuestionIdentifier>output</QuestionIdentifier><FreeText/></Answer></QuestionFormAnswers>'}], 'ResponseMetadata': {'RequestId': '1b7fdfea-366b-4589-9a95-806a5e5c7894', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '1b7fdfea-366b-4589-9a95-806a5e5c7894', 'content-type': 'application/x-amz-json-1.1'

In [25]:
# vizier_retriever.pickle_this(assignment_batch, 'hit_data/prod_b9')

In [36]:
updated_hits = [vizier.amt.client.get_hit(HITId=hit['HITId']) for hit in hit_batch]

In [47]:
n_still_avail = pd.Series([uh['HIT']['NumberOfAssignmentsAvailable'] for uh in updated_hits])

In [51]:
n_still_avail.value_counts()

0    12
2     8
1     8
3     2
dtype: int64

In [120]:
client = boto3.client('logs')

In [126]:
logs = client.describe_log_groups()

In [45]:
# for log in logs["logGroups"]:
#     print(log['logGroupName'])

In [25]:
# client.describe_log_streams(logGroupName='/opt/vision-pictionary/log/output-0.log')

## 3.1 s3 sync experiment

In [138]:
import boto3

In [144]:
s3 = boto3.resource('s3')
s3client = boto3.client('s3', )

In [198]:
vision_mturk_bucket = 'ai2-vision-mturk'
project_folder = 'iconary/'
project_subfolder = 'hits/'

In [ ]:
s3client.create_multipart_upload()

In [199]:
def create_folder(client, **kwargs):
    response = client.put_object(
        Bucket=kwargs.get('bucket', ''),
        Body='',
        Key=kwargs.get('new_folder', ''))
    return response

In [200]:
create_folder(s3client, bucket=vision_mturk_bucket, new_folder=project_folder + project_folder)

{'ETag': '"d41d8cd98f00b204e9800998ecf8427e"',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '0',
   'date': 'Fri, 18 May 2018 21:29:10 GMT',
   'etag': '"d41d8cd98f00b204e9800998ecf8427e"',
   'server': 'AmazonS3',
   'x-amz-id-2': 'Dks+mbNHlew3Uo//ElBPdgAFMIhSrgIhVVso/cFlgTDDYTvjlEGnhS6LFqiaZo85fjeVknxZdgw=',
   'x-amz-request-id': '555EADB56068AD65'},
  'HTTPStatusCode': 200,
  'HostId': 'Dks+mbNHlew3Uo//ElBPdgAFMIhSrgIhVVso/cFlgTDDYTvjlEGnhS6LFqiaZo85fjeVknxZdgw=',
  'RequestId': '555EADB56068AD65',
  'RetryAttempts': 0}}

In [177]:
response = s3client.list_buckets(key=vision_mturk_bucket)

In [165]:
all_objects = s3client.list_objects(Bucket=vision_mturk_bucket, Prefix='dipart-10') 

In [158]:
def folders(client, bucket, prefix=''):
    paginator = client.get_paginator('list_objects')
    for result in paginator.paginate(Bucket=bucket, Prefix=prefix, Delimiter='/'):
        for prefix in result.get('CommonPrefixes', []):
            yield prefix.get('Prefix')

In [190]:
pdat = pickle.dumps('test pickled data')
test_data = io.BytesIO(pdat)

In [191]:
import io

In [ ]:
test_data

In [192]:
s3client.upload_fileobj(test_data, vision_mturk_bucket, project_folder + 'test.pkl')

In [196]:
with open('/Users/schwenk/Downloads/test.pkl', 'rb') as f:
    dd = pickle.load(f)

In [183]:
gen_folders = folders(s3client,  vision_mturk_bucket)
list(gen_folders)

['dipart-10/',
 'dqa_matching_images/',
 'iconary/',
 'tqa-matching/',
 'tqa-matching2/']

In [170]:
gen_subfolders = folders(s3client, vision_mturk_bucket, prefix='tqa-matching')
list(gen_subfolders)

['tqa-matching/', 'tqa-matching2/']

In [148]:
for bucket in response["Buckets"]:
    print(bucket['Name'])

ai2
ai2-ackbar
ai2-aristo-demo-cloudfront-logs
ai2-aristo-diagramparses
ai2-aristo-elasticsearch
ai2-aristo-evaluations
ai2-aristo-experiments
ai2-aws-billing
ai2-aws-configs
ai2-backup
ai2-beaker
ai2-certify
ai2-dericka
ai2-ek-test
ai2-environments
ai2-euclid
ai2-euclid-im2txt
ai2-euclid-public
ai2-euclid-tasks
ai2-euclid-turk
ai2-evaluation-cli
ai2-evaluation-framework-backups
ai2-fluent-logs
ai2-git-annex
ai2-incubator
ai2-infrastructure
ai2-lambda
ai2-levan
ai2-logs
ai2-longterm-glacier
ai2-matthewp
ai2-michaels
ai2-michalg
ai2-misc
ai2-mohiti
ai2-openie
ai2-ops-cloudtrail
ai2-ops-transfer
ai2-oqa
ai2-oyvindt
ai2-packages
ai2-papertrail-backup
ai2-public-websites
ai2-research-git-annex
ai2-s2
ai2-s2-analytics
ai2-s2-anansi
ai2-s2-athena-elb-log
ai2-s2-banner-ads
ai2-s2-blacklisted-pdfs
ai2-s2-brendanr
ai2-s2-build
ai2-s2-citeomatic
ai2-s2-citeseer
ai2-s2-cloudfront-access-logs
ai2-s2-crawler
ai2-s2-data
ai2-s2-data-incremental
ai2-s2-datateam
ai2-s2-deepfigures
ai2-s2-dev
ai2-s2-de

# 4. approve

In [32]:
client_params = {
    'profile_name': 'mturk_vision',
    'in_production': True,
    'n_threads': 1,
    's3_base_path': '',
}
vizier_retriever = Vizier(**client_params)

Account balance is: $2616.14


In [33]:
# dels_resp = vizier.force_delete_hits(all_sbx_hits)

In [135]:
# _ = vizier_retriever.set_hits_reviewing(all_sbx_hits)

In [136]:
_ = vizier_retriever.approve_hits(all_sbx_hits)

In [105]:
# vizier.pickle_this(all_hits, 'all_prod_hits_5_21.pkl')

In [137]:
# vizier_retriever.delete_hits(all_sbx_hits)

In [20]:
trial_hit_batch = [h for h in all_hits if h['Title'] == static_params['Title']]

In [21]:
len(trial_hit_batch)

400

In [73]:
# mt_client.approve_assignments()

# 5. assign qaulification

In [16]:
workers_to_qualify = pd.read_csv('first_qualified_workers.csv', header=None)[0].tolist()

In [17]:
len(workers_to_qualify)

184

In [66]:
iconary_qualification = {
    'Name': 'Iconary qualifification',
    'Keywords': 'ai2, vision, iconary, pictionary',
    'Description': 'Qualified to play Iconary as both drawer and guesser.',
    'QualificationTypeStatus': 'Active',
}

In [67]:
qual_resp = vizier.create_qualification(**iconary_qualification)

In [68]:
with open('iconary_qual', 'wb') as f:
    pickle.dump(qual_resp, f)

In [69]:
qual_id = qual_resp['QualificationType']['QualificationTypeId']

In [107]:
qual_id

'3Z1HL5WC7LSXUFW49BUXR7ZP1IDH6M'

In [5]:
my_worker_id = 'ATRVCA186WV1B'

In [61]:
underperforming_workers = ['A29VL3MZE7YPBZ', 'A2ZZW6KME1FUDU', 'AGX171Q54YXPI']

# 6. message workers

In [85]:
message_subject = 'New related task- Viconary'
message_body = """
Hello,

Thanks for your effort on the last round of viconary.

The last batch of drawings were an improvement, but we still think many drawings can be even simpler.

We've made some new examples to explain what we're going for- http://viconary.allenai.org/examples.html

These examples are also linked from the instructions page, if you'd like to refer to them there.

We're submitting another test batch this afternoon if you'd like to give these a try.

-Vision AI2
"""

In [86]:
test_worker_id = pd.read_pickle('viconary_first_test_wids.pkl').tolist()

In [22]:
# with open('active_workers.pkl', 'rb') as f:
#     active_workers = pickle.load(f)
# len(active_workers)

In [88]:
msg_resp = vizier.message_workers(test_worker_id, message_subject, message_body)